⊕ [Tutorial: Python in 10 Minutes - ArangoDB database](https://www.arangodb.com/tutorials/tutorial-python/)


In [1]:
from pyArango.connection import *
conn = Connection(username="root", password="")

When this code executes, it initializes the server connection on the conn variable. By default, pyArango attempts to establish a connection to http://127.0.0.1:8529. That is, it wants to initialize a remote connection to your local host on port 8529. If you are hosting ArangoDB on a different server or use a different port, you need to set these options when you instantiate the Connection class.



In [3]:
# db = conn.createDatabase(name="school")
db = conn["school"]
db

ArangoDB database: school

In [4]:
>>> studentsCollection = db.createCollection(name="Students")
>>> db["Students"]

ArangoDB collection name: Students, id: 13724, type: document, status: loaded

## 创建集合
ArangoDB将文档和边缘分组到集合中。这类似于关系数据库中表的概念，但关键的区别是集合是无模式的。

在pyArango中，您可以通过调用createCollection()给定数据库上的方法来创建集合。例如，在上一节中，您创建了一个school数据库。您可能希望为该学生提供该数据库的集合。

In [6]:
# >>> studentsCollection = db.createCollection(name="Students")
>>> db["Students"]

ArangoDB collection name: Students, id: 13724, type: document, status: loaded

## 创建文档
设置数据库和集合后，您可以开始向它们添加数据。继续与Relational数据库进行比较，其中集合是表，文档是该表上的一行。但是，与行不同，文档是无模式的。您可以包含应用程序所需的任何值排列。

例如，将学生添加到集合中：

In [20]:
>>> doc1 = studentsCollection.createDocument()
>>> doc1["name"] = "John Smith"
>>> doc1

ArangoDoc '_id: None, _key: None, _rev: None': <store: {'name': 'John Smith'}>

In [8]:
>>> doc2 = studentsCollection.createDocument()
>>> doc2["firstname"] = "Emily"
>>> doc2["lastname"] = "Bronte"
>>> doc2

ArangoDoc '_id: None, _key: None, _rev: None': <store: {'firstname': 'Emily', 'lastname': 'Bronte'}>

该文档显示_id为“无”，因为您尚未将其保存到ArangoDB。这意味着变量存在于Python代码中，但不存在于数据库中。ArangoDB _id通过将集合名称与值配对来构造值_key。它还为您处理分配，您只需设置密钥并保存文档。



In [21]:
>>> doc1._key = "johnsmith"
>>> doc1.save()
>>> doc1

ArangoDoc '_id: Students/johnsmith, _key: johnsmith, _rev: _Yq3PTwu--_': <store: {'name': 'John Smith'}>

In [22]:
studentsCollection["johnsmith"]

ArangoDoc '_id: Students/johnsmith, _key: johnsmith, _rev: _Yq3PTwu--_': <store: {'name': 'John Smith'}>

您可能希望从数据库中删除文档。这可以通过该delete()方法完成。

In [23]:
>>> tom = studentsCollection["johnsmith"]
>>> tom.delete()
# >>> studentsCollection["johnsmith"]

In [10]:
students = [('Oscar', 'Wilde', 3.5), ('Thomas', 'Hobbes', 3.2), 
    ('Mark', 'Twain', 3.0), ('Kate', 'Chopin', 3.8), ('Fyodor', 'Dostoevsky', 3.1), 
    ('Jane', 'Austen',3.4), ('Mary', 'Wollstonecraft', 3.7), ('Percy', 'Shelley', 3.5), 
    ('William', 'Faulkner', 3.8), ('Charlotte', 'Bronte', 3.0)]
for (first, last, gpa) in students:
    doc = studentsCollection.createDocument()
    doc['name'] = "%s %s" % (first, last)
    doc['gpa'] = gpa 
    doc['year'] = 2017
    doc._key = ''.join([first, last]).lower() 
    doc.save()

In [12]:
# 访问数据库中的文档。最简单的方法是使用_key值。
def report_gpa(document):
    print("Student: %s" % document['name'])
    print("GPA:     %s" % document['gpa'])
kate = studentsCollection['katechopin']
report_gpa(kate)

Student: Kate Chopin
GPA:     3.8


In [13]:
# 当您从ArangoDB中读取文档到您的应用程序时，您将创建该文档的本地副本。然后，您可以对文档进行操作，对其进行任何更改，然后使用该save()方法将结果推送到数据库。
def update_gpa(key, new_gpa):
   doc = studentsCollection[key]
   doc['gpa'] = new_gpa
   doc.save()

In [24]:
# 您可能希望对给定集合中的所有文档进行操作。使用该fetchAll()方法，您可以检索并迭代文档列表。例如，假设这是学期结束，你想知道哪些学生的平均成绩高于3.5：
def top_scores(col, gpa):
   print("Top Soring Students:")
   for student in col.fetchAll():
      if student['gpa'] >= gpa:
         print("- %s" % student['name'])
top_scores(studentsCollection, 3.5)

Top Soring Students:
- Mary Wollstonecraft
- Kate Chopin
- Percy Shelley
- William Faulkner
- Oscar Wilde


## AQL用法
除了上面显示的Python方法之外，ArangoDB还提供了一种查询语言（称为AQL），用于检索和修改数据库上的文档。在pyArango中，您可以使用该AQLQuery()方法发出这些查询。

例如，假设您要检索ArangoDB中所有文档的密钥：

In [25]:
aql = "FOR x IN Students RETURN x._key"
queryResult = db.AQLQuery(aql, rawResults=True, batchSize=100)
for key in queryResult:
   print(key)

marywollstonecraft
katechopin
percyshelley
fyodordostoevsky
marktwain
williamfaulkner
oscarwilde
charlottebronte
thomashobbes
janeausten


在上面的示例中，该AQLQuery()方法将AQL查询作为参数，并带有两个附加选项：

* rawResults 定义是否需要查询返回的实际结果。
* batchSize 当查询返回的结果多于给定值时，pyArango驱动程序会自动请求新批次。

请记住，文件的顺序不能保证。如果您需要按特定顺序排列结果，请在AQL查询中添加sort子句。

⊕ [Graph | Example AQL Queries for Graphs | Cookbook | ArangoDB Documentation](https://www.arangodb.com/docs/stable/cookbook/graph-example-actors-and-movies.html)


During this solution we will be using arangosh to create and query the data. All the AQL queries are strings and can simply be copied over to your favorite driver instead of arangosh.

Create a Test Dataset in arangosh: ...

示例是使用nodejs编写的.

## 使用AQL插入文档
与上面的文档创建类似，您也可以使用AQL将文档插入ArangoDB。这是通过INSERT使用bindVars该AQLQuery()方法选项的语句完成的。

In [26]:
>>> doc = {'_key': 'denisdiderot', 'name': 'Denis Diderot', 'gpa': 3.7}
>>> bind = {"doc": doc}
>>> aql = "INSERT @doc INTO Students LET newDoc = NEW RETURN newDoc"
>>> queryResult = db.AQLQuery(aql, bindVars=bind)

In [27]:
queryResult[0]

ArangoDoc '_id: Students/denisdiderot, _key: denisdiderot, _rev: _Yq3Y6fK--_': <store: {'gpa': 3.7, 'name': 'Denis Diderot'}>

## 使用AQL更新文档
如果数据库中已存在文档并且您想要修改该文档中的数据，则可以使用该UPDATE语句。例如，假设您在CSV文件中收到学生的更新成绩点平均值。

首先，检查其中一个学生的GPA以查看旧值：

In [28]:
db["Students"]["katechopin"]

ArangoDoc '_id: Students/katechopin, _key: katechopin, _rev: _Yq3KVZK--_': <store: {'name': 'Kate Chopin', 'gpa': 3.8, 'year': 2017}>

In [30]:
# 然后遍历文件，更新每个学生的GPA：
with open("grades.csv", "r") as f:
   grades = f.read().split(',')
for key,gpa in grades.items():
   doc = {"gpa": float(gpa)}
   bind = {"doc": doc, "key": key}
   aql = "UPDATE @key WITH @doc IN Stdents LET pdated NEW RETRN updated"
   db.AQLQuery(aql, bindVars=bind)

# 最后，再次检查学生的GPA。
db["Students"]["katechopin"]

FileNotFoundError: [Errno 2] No such file or directory: 'grades.csv'